# Some machine learning examples

In [ ]:
import numpy as np
from matplotlib import pyplot as plt
from matplotlib import cm
import seaborn as sns
from IPython.display import HTML
from celluloid import Camera
%matplotlib notebook
sns.set()

## Example 1

### Generating data

In [ ]:
np.random.seed(100)

# Parameters
num_dims = 1
num_data_points = 50
data_size = (num_data_points, num_dims)
x_scale = 20.
x_offset = -10.
noise_scale = 5.
weights_scale = np.array([10., 6.])
weights_offset = np.array([-5., -3.])

weights = (np.random.random((num_dims + 1,)) * weights_scale) + weights_offset

In [ ]:
# Generating the data
X = (np.random.random(data_size) * x_scale) + x_offset

Xa = np.concatenate((np.ones((num_data_points, 1)), X), 1)
y = (Xa @ weights) + np.random.normal(0., noise_scale, (num_data_points,))

### Visualization

In [ ]:
fig_data = plt.figure()
plt.xlabel('x')
plt.xlim((-10, 10))
plt.ylabel('y')
plt.ylim((-10, 10))
plt.title('Data')
ax_data = sns.scatterplot(x=Xa[:, 1], y=y)
sns.lineplot(x=Xa[:, 1], y=Xa @ weights, color='black', ax=ax_data)
plt.show()

### Algorithm

In [ ]:
# Parameters
alpha = 5e-5
tolerance = 1.5
max_iterations = 1000

In [ ]:
# Algorithm
trace_w = []

iteration = 0
last_loss = np.infty
w = (np.random.random((num_dims + 1,)) * weights_scale) + weights_offset
trace_w.append(w.copy())
yhat = Xa @ w
delta = y - yhat
loss = delta @ delta
while np.abs(last_loss - loss) > tolerance:
    iteration += 1
    if iteration > max_iterations:
        iteration -= 1
        break
    nabla_w = -2 * (delta @ Xa)
    w -= alpha * nabla_w
    trace_w.append(w.copy())
    yhat = Xa @ w
    delta = y - yhat
    last_loss = loss
    loss = delta @ delta
print("Finished after " + str(iteration) + " iterations.")

### Visualizing the algorithm in action

In [ ]:
%%capture
# Creating animated visualization
fig = plt.figure()
camera = Camera(fig)
plt.xlabel('x')
plt.ylabel('y')

plt.title('Linear Regression')

for i in range(iteration):
    w = trace_w[i]
    ax = sns.scatterplot(x=Xa[:, 1], y=y, legend=False)
    sns.lineplot(x=Xa[:, 1], y=Xa @ w, color='black', legend=False, ax=ax)
    plt.xlim((-10, 10))
    plt.ylim((-10, 10))
    camera.snap()

animation = camera.animate()
ax = sns.scatterplot(x=Xa[:, 1], y=y, legend=False)
plt.show()

In [ ]:
# Playing the animation
HTML(animation.to_jshtml())

## Example 2

### Generating data

In [ ]:
np.random.seed(100)

# Parameters
num_dims = 2
num_data_points = 100
data_size = (num_data_points, num_dims)
x_scale = np.array([[20., 20.]])
x_offset = np.array([[-10., -10.]])
noise_scale = 8.
weights_scale = np.array([10., 10.])
weights_offset = np.array([-5., -5.])

weights = (np.random.random((num_dims,)) * weights_scale) + weights_offset

In [ ]:
# Generating the data
X = (np.random.random(data_size) * np.tile(x_scale, (num_data_points, 1))) + np.tile(x_offset, (num_data_points, 1))

Xa = np.concatenate((np.ones((num_data_points, 1)), X[:, :-1]), 1)
z = (Xa @ weights) + np.random.normal(0., noise_scale, (num_data_points,))
classes = (z > X[:, -1]).astype(int)
y = (2 * classes) - 1

### Visualization

In [ ]:
fig_data = plt.figure()
plt.xlabel('x')
plt.xlim((-10, 10))
plt.ylabel('y')
plt.ylim((-10, 10))
plt.title('Data')
ax = sns.scatterplot(x=X[:, 0], y=X[:, 1], hue=classes, legend=False)
sns.lineplot(x=Xa[:, 1], y=Xa @ weights, color='black', legend=False, ax=ax)
plt.show()

### Algorithm

In [ ]:
# Parameters
alpha = 1e-4
tolerance = 1e-2
max_iterations = 1000

In [ ]:
# Algorithm
trace_w = []
trace_b = []

iteration = 0
last_loss = np.infty
w = 10 * np.random.random((num_dims,)) - 5
b = np.random.random()
trace_w.append(w.copy())
trace_b.append(b)
zhat = (X @ w) + b
mhat = y * zhat
p1 = (mhat < 0)
p2 = np.logical_and(np.logical_not(p1), (mhat < 1))
loss = -np.sum(mhat[p1])
while np.abs(last_loss - loss) > tolerance:
    iteration += 1
    if iteration > max_iterations:
        iteration -= 1
        break
    a1 = -y[None, p1]
    a2 = -y[None, p2] - zhat[None, p2]
    nabla_w = np.squeeze((a1 @ X[p1, :]) + (a2 @ X[p2, :]))
    nabla_b = np.squeeze(np.sum(a1) + np.sum(a2))
    w -= alpha * nabla_w
    b -= alpha * nabla_b
    trace_w.append(w.copy())
    trace_b.append(b)
    zhat = (X @ w) + b
    mhat = y * zhat
    p1 = (mhat < 0)
    p2 = np.logical_and(np.logical_not(p1), (mhat < 1))
    last_loss = loss
    loss = -np.sum(mhat[p1])
print("Finished after " + str(iteration) + " iterations.")

### Visualizing the algorithm in action

In [ ]:
%%capture
# Creating animated vissualization
fig = plt.figure()
camera = Camera(fig)
plt.xlabel('x')
plt.ylabel('y')

plt.title('SVM Classification')

for i in range(iteration):
    w = trace_w[i]
    b = trace_b[i]
    ax = sns.scatterplot(x=X[:, 0], y=X[:, 1], hue=classes, legend=False)
    sns.lineplot(x=np.array([-10, 10]), y=-((np.array([-10, 10]) * w[0]) + b) / w[1], color='black', legend=False, ax=ax)
    plt.xlim((-10, 10))
    plt.ylim((-10, 10))
    camera.snap()

animation = camera.animate()
ax = sns.scatterplot(x=X[:, 0], y=X[:, 1], hue=classes, legend=False)
plt.show()

In [ ]:
# Playing the animation
HTML(animation.to_jshtml())

## Example 3

### Generating the data

In [ ]:
np.random.seed(100)

# Parameters
num_dims = 2
gen_k = 5
gen_class_centre_means = 20. * np.random.random((gen_k, num_dims)) - 10.
gen_class_centre_covs = ((5. * np.random.random((gen_k, 1))) @ np.eye(num_dims)[:, None, :]).transpose(1, 0, 2)
gen_num_class_points = 40

num_data_points = gen_k * gen_num_class_points

In [ ]:
# Generating the data
X = np.zeros((num_data_points, num_dims))
y = np.zeros((num_data_points,), dtype=int)
for i in range(gen_k):
    ind_slice = (i * gen_num_class_points) + np.arange(gen_num_class_points, dtype=int)
    X[ind_slice] = np.random.multivariate_normal(gen_class_centre_means[i], gen_class_centre_covs[i], (gen_num_class_points,))
    y[ind_slice] = i

### Visualization

In [ ]:
fig_data = plt.figure()
plt.xlabel('x')
plt.xlim((-10, 10))
plt.ylabel('y')
plt.ylim((-10, 10))
plt.title('Data')
ax = sns.scatterplot(x=gen_class_centre_means[:, 0], y=gen_class_centre_means[:, 1], hue=np.arange(gen_k), marker='x', legend=False)
sns.scatterplot(x=X[:, 0], y=X[:, 1], hue=y, legend=False, ax=ax)
plt.show()

### Algorithm

In [ ]:
# Parameters
k = 5

In [ ]:
# Algorithm
trace_centres = []
trace_y = []

iteration = 0
last_y = np.zeros((num_data_points,), dtype=int)
y = np.random.randint(0, k, (num_data_points,))
trace_y.append(y.copy())
centres = np.zeros((k, num_dims))
for i in np.arange(k):
    centres[i, :] = np.average(X[y == i, :], 0)
trace_centres.append(centres.copy())

while np.any(y != last_y):
    iteration += 1
    distances_sq = np.sum((np.tile(X[:, :, None], (1, 1, k)) - np.tile(centres[:, :, None].transpose((2, 1, 0)), (num_data_points, 1, 1))) ** 2, 1)
    last_y = y
    y = np.argmin(distances_sq, 1)
    trace_y.append(y.copy())
    for i in np.arange(k):
        cluster_data_points = X[y == i, :]
        if cluster_data_points.shape[0] > 0:
            centres[i, :] = np.average(cluster_data_points, 0)
        else:
            centres[i, :] = X[np.random.randint(0, num_data_points), :]
    trace_centres.append(centres.copy())
print("Finished after " + str(iteration) + " iterations.")

### Visualizing the algorithm in action

In [ ]:
%%capture
# Creating animated vissualization
fig = plt.figure()
camera = Camera(fig)
plt.xlabel('x')
plt.ylabel('y')

plt.title('k-Means clustering')

for i in range(iteration):
    y = trace_y[i]
    centres = trace_centres[i]
    ax = sns.scatterplot(x=centres[:, 0], y=centres[:, 1], hue=np.arange(gen_k), marker='x', legend=False)
    sns.scatterplot(x=X[:, 0], y=X[:, 1], hue=y, legend=False, ax=ax)
    plt.xlim((-10, 10))
    plt.ylim((-10, 10))
    camera.snap()

animation = camera.animate()
plt.show()

In [ ]:
# Playing the animation
HTML(animation.to_jshtml())

## Example 4

### Specifying an environment

In [ ]:
np.random.seed(100)

# Environment shape and artifcats
actions = np.array([[-1,  0],
                    [ 0, -1],
                    [ 1,  0],
                    [ 0,  1]], dtype=int)
rt = 0
up = 1
lf = 2
dn = 3

grid = np.array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 1],
                 [1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1],
                 [1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1],
                 [1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1],
                 [1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1],
                 [1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1],
                 [1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1],
                 [1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1],
                 [1, 0, 1, 1, 1, 1, 1, 1, 0, 4, 1],
                 [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
                 [1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1]], dtype=int)
path = 0
wall = 1
strt = 2
goal = 3
objt = 4

symbol_rewards = np.array([0., -1., 0., 10., 3.])
out_reward = -1.

symbol_terminal = np.array([False, False, False, True, False], dtype=bool)

In [ ]:
# Environment actions
episode_grid = None

def gridworld_init():
    global episode_grid
    episode_grid = grid
    return np.concatenate(np.where(episode_grid == strt))

def gridworld_step(s, a):
    global episode_grid
    sprime = s + actions[a]
#    print("si: " + str(sprime))
    if np.any(sprime < 0) or np.any(sprime >= episode_grid.shape):
        r = out_reward
        sprime = s
    else:
        r = symbol_rewards[episode_grid[tuple(sprime.tolist())]]
    if episode_grid[tuple(sprime.tolist())] == wall:
        sprime = s
    elif episode_grid[tuple(sprime.tolist())] == objt:
        episode_grid[tuple(sprime.tolist())] = path
    return (sprime, r)

def gridworld_is_terminal(s):
    return symbol_terminal[episode_grid[s[0], s[1]]]

### Visualization

In [ ]:
# Visualization utilities
cmap = cm.RdBu
cmap.set_bad('black', 1.)

vis_filter = np.array([1., np.nan, 1., 1., 1.])[grid]

In [ ]:
# Visualization
vis_map = 0. * vis_filter
masked_array = np.ma.array(vis_map, mask=np.isnan(vis_map))

fig, ax = plt.subplots(1, 1)

ax.imshow(masked_array, cmap=cmap, vmin=-40., vmax=40.)

ax.axis('off')

start_pos = (np.concatenate(np.where(grid == strt)))
start_glyph = plt.scatter(start_pos[1], start_pos[0], marker="$S$", color='green')
goal_pos = (np.concatenate(np.where(grid == goal)))
goal_glyph = plt.scatter(goal_pos[1], goal_pos[0], marker="$G$", color='blue')
pizza_pos = (np.concatenate(np.where(grid == objt)))
pizza_glyph = plt.scatter(pizza_pos[1], pizza_pos[0], marker="$P$", color='red')

plt.show()

### Algorithm

In [ ]:
# Parameters
eps = 0.2
alpha = 0.1
gamma = 0.9

In [ ]:
# Algorithm
Q = np.zeros((grid.shape[0], grid.shape[1], actions.shape[0]))
MS = np.zeros((grid.shape[0], grid.shape[1]), dtype=bool)
MSA = np.zeros((grid.shape[0], grid.shape[1], actions.shape[0]), dtype=bool)
MSp = np.zeros((grid.shape[0], grid.shape[1], actions.shape[0], 2), dtype=int)
MR = np.zeros((grid.shape[0], grid.shape[1], actions.shape[0]))
Mus = 10

def randargmax(vec):
    return np.random.choice(np.where(vec == np.max(vec))[0])

def Qrow(s):
    return Q[s[0], s[1], :]

def epsilon_greedy(s, epsilon):
    return randargmax(Qrow(s)) if np.random.random() > epsilon else np.random.randint(Qrow(s).shape[0])

def V(s):
    return np.max(Qrow(s))

def qlearning_episode(record=False, max_steps=100):
    global Q, MS, MSA, MSp, MR
    step = 0
    s = gridworld_init()
    if record and step < max_steps:
        trace_Q = [Q.copy()]
        trace_s = [s]
    else:
        trace_Q = []
        trace_s = []
    trace_a = []
    while not gridworld_is_terminal(s):
        step += 1
        MS[s[0], s[1]] = True
        a = epsilon_greedy(s, eps)
        MSA[s[0], s[1], a] = True
        (sprime, r) = gridworld_step(s, a)
        MSp[s[0], s[1], a, :] = sprime
        MR[s[0], s[1], a] = r
        td_error = r - Q[s[0], s[1], a]
        if not gridworld_is_terminal(sprime):
            td_error += gamma * V(sprime)
        Q[s[0], s[1], a] += alpha * td_error
        MSI = np.where(MS)
        num_MS = MSI[0].shape[0]
        for i in range(Mus):
            si = np.random.randint(num_MS)
            s_m = np.array([MSI[0][si], MSI[1][si]])
            MsAI = np.where(MSA[s_m[0], s_m[1], :])
            num_MsA = MsAI[0].shape[0]
            ai = np.random.randint(num_MsA)
            a_m = MsAI[0][ai]
            sprime_m = MSp[s_m[0], s_m[1], a_m, :]
            r_m = MR[s_m[0], s_m[1], a_m]
            td_error_m = r_m - Q[s_m[0], s_m[1], a_m]
            if not gridworld_is_terminal(sprime_m):
                td_error_m += gamma * V(sprime_m)
            Q[s_m[0], s_m[1], a_m] += alpha * td_error_m
        s = sprime
        if record and step < max_steps:
            trace_Q.append(Q.copy())
            trace_s.append(s)
            trace_a.append(a)
    return (step, trace_Q, trace_s, trace_a)

### Execution

#### Episode (iteration) 1

##### Execution

In [ ]:
Q = np.random.random((grid.shape[0], grid.shape[1], actions.shape[0]))

(num_steps, trace_Q, trace_s, trace_a) = qlearning_episode(True)
print("Episode 1 finished after " + str(num_steps) + " steps.")

##### Visualizing the algorithm in action

In [ ]:
%%capture
# Creating animated visualization
fig, ax = plt.subplots(1, 1)
camera = Camera(fig)

Q = trace_Q[0]
s = trace_s[0]
a = trace_a[0]

vis_map = np.max(Q, 2) * vis_filter
masked_array = np.ma.array(vis_map, mask=np.isnan(vis_map))

ax.imshow(masked_array, cmap=cmap, vmin=-10., vmax=10.)
ax.grid(True)
agent_glyph = plt.scatter(s[1], s[0], marker="o", color=[0.8, 0.8, 0.8], edgecolors='black')
camera.snap()
for i in range(1, len(trace_Q)):
    a = trace_a[i - 1]
    
    ax.imshow(masked_array, cmap=cmap, vmin=-10., vmax=10.)
    ax.grid(True)
    sa = s + actions[a, :]
    agent_glyph = plt.scatter(sa[1], sa[0], marker="o", color=[0.8, 0.8, 0.8], edgecolors='yellow')
    camera.snap()
    
    Q = trace_Q[i]
    s = trace_s[i]
    
    vis_map = np.max(Q, 2) * vis_filter
    masked_array = np.ma.array(vis_map, mask=np.isnan(vis_map))

    ax.imshow(masked_array, cmap=cmap, vmin=-10., vmax=10.)
    ax.grid(True)
    agent_glyph = plt.scatter(s[1], s[0], marker="o", color=[0.8, 0.8, 0.8], edgecolors='black')
    camera.snap()
animation = camera.animate()
plt.show()

In [ ]:
# Playing the animation
HTML(animation.to_jshtml())

#### Episode (iteration) 2 to 1000

##### Execution

In [ ]:
for i in range(2, 1000):
    (num_steps, trace_Q, trace_s, trace_a) = qlearning_episode(False)
    print("Episode " + str(i) + " finished after " + str(num_steps) + " steps.")
(num_steps, trace_Q, trace_s, trace_a) = qlearning_episode(True)
print("Episode 1000 finished after " + str(num_steps) + " steps.")

##### Visualizing the algorithm in action for episode 1000

In [ ]:
%%capture
# Creating animated visualization
fig, ax = plt.subplots(1, 1)
camera = Camera(fig)

Q = trace_Q[0]
s = trace_s[0]
a = trace_a[0]

vis_map = np.max(Q, 2) * vis_filter
masked_array = np.ma.array(vis_map, mask=np.isnan(vis_map))

ax.imshow(masked_array, cmap=cmap, vmin=-10., vmax=10.)
ax.set_xlim((-1.5, grid.shape[0] + 0.5))
ax.set_ylim((grid.shape[1] + 0.5, -1.5))
ax.axis('off')
agent_glyph = plt.scatter(s[1], s[0], marker="o", color=[0.8, 0.8, 0.8], edgecolors='black')
camera.snap()
for i in range(1, len(trace_Q)):
    a = trace_a[i - 1]
    
    ax.imshow(masked_array, cmap=cmap, vmin=-10., vmax=10.)
    ax.set_xlim((-1.5, grid.shape[0] + 0.5))
    ax.set_ylim((grid.shape[1] + 0.5, -1.5))
    ax.axis('off')
    sa = s + actions[a, :]
    agent_glyph = plt.scatter(sa[1], sa[0], marker="o", color=[0.8, 0.8, 0.8], edgecolors='yellow')
    camera.snap()
    
    Q = trace_Q[i]
    s = trace_s[i]
    
    vis_map = np.max(Q, 2) * vis_filter
    masked_array = np.ma.array(vis_map, mask=np.isnan(vis_map))

    ax.imshow(masked_array, cmap=cmap, vmin=-10., vmax=10.)
    ax.set_xlim((-1.5, grid.shape[0] + 0.5))
    ax.set_ylim((grid.shape[1] + 0.5, -1.5))
    ax.axis('off')
    agent_glyph = plt.scatter(s[1], s[0], marker="o", color=[0.8, 0.8, 0.8], edgecolors='black')
    camera.snap()
animation = camera.animate()
plt.show()

In [ ]:
# Playing the animation
HTML(animation.to_jshtml())